div align="center">
    <img src="https://socialify.git.ci/julep-ai/julep/image?description=1&descriptionEditable=Build%20AI%20agents%20and%20workflows%20with%20a%20simple%20API&font=Source%20Code%20Pro&logo=https%3A%2F%2Fraw.githubusercontent.com%2Fjulep-ai%2Fjulep%2Fdev%2F.github%2Fjulep-logo.svg&owner=1&pattern=Solid&stargazers=1&theme=Auto" alt="julep" width="640" height="320" />
</div>

## Task: Travel Itinerary Assistant with Weather and Wikipedia Integrations

### Overview

The Travel Itinerary Assistant helps users plan a travel itinerary that takes into account current weather conditions and local tourist attractions. By integrating data from Wikipedia for tourist attractions and using a weather API for real-time weather updates, the tool provides a comprehensive travel plan tailored to each location. The generated itinerary suggests appropriate activities based on the weather, enhancing the overall travel experience.

### Task Flow

1. **User Input**
   - User provides a list of desired travel locations.
   - Each location is processed individually to gather the required data.

2. **Weather Data Retrieval**
   - Fetch current weather data for each location using a weather API.
   - Extract relevant weather details, such as temperature, weather condition, and recommendations.

3. **Tourist Attractions Lookup**
   - Use Wikipedia to search for the top tourist attractions for each location.
   - The query format used is: `"<location> tourist attractions"`.
   - Retrieve and compile a list of popular tourist spots and landmarks.

4. **Data Evaluation and Integration**
   - Combine weather data and tourist attractions into a unified list for each location.
   - Format the data into a structured tuple: `(location, weather, attractions)`.

5. **Itinerary Generation**
   - Create a detailed travel itinerary based on:
     - Current weather conditions (e.g., sunny, rainy, cloudy).
     - Top tourist attractions for each location.
     - Suggested activities categorized as indoor or outdoor based on weather.

### Key Features

- **Multi-location Travel Planning**: Handles multiple destinations simultaneously, offering a consolidated travel plan.
- **Real-time Weather Data**: Leverages weather APIs to provide up-to-date weather conditions.
- **Tourist Attraction Discovery**: Integrates Wikipedia to find and recommend popular attractions.
- **Intelligent Itinerary Suggestions**: Suggests indoor or outdoor activities based on the weather.
- **Comprehensive Itinerary Output**: Combines weather and tourist data into a user-friendly travel plan.

### Output

- A detailed travel itinerary for each location
- Curated, up-to-date information gathered from weather searches and Wikipedia

```plaintext

+----------------+     +--------------------------+     +--------------------------+     +------------------------------+     +-------------------------+
|   User Input   |     | Weather Data Retrieval   |     | Tourist Attractions      |     | Data Evaluation & Integration|     | Itinerary Generation    |
|    (List of    | --> |  (Weather API)           | --> | Lookup (Wikipedia)       | --> |    (Combine Weather &        | --> |  (Generate Suggested    |
|    Locations)  |     |                          |     |                          |     |   Attractions Data)          |     |    Activities/Plan)     |
+----------------+     +--------------------------+     +--------------------------+     +------------------------------+     +-------------------------+
       |                              |                             |                                   |                                  |
       |                              |                             |                                   |                                  |
       v                              v                             v                                   v                                  v
Location 1, Location 2, ...   Fetch weather for each       Search Wikipedia for           Combine weather data and             Create itinerary with
Each location processed       location individually,       "<location> tourist            tourist attractions into              suggested activities
individually for              extracting temp.,            attractions", retrieve         a structured tuple:                    based on weather and
weather data.                 conditions, &                top spots.                     (location, weather,                       attractions.
                               recommendations.                                             attractions).
```




## Implementation

To recreate the notebook and see the code implementation for this task, you can access the Google Colab notebook using the link below:

<a target="_blank" href="https://colab.research.google.com/github/julep-ai/julep/blob/dev/cookbooks/04-TripPlanner_With_Weather_And_WikiInfo.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

### Additional Information

For more details about the task or if you have any questions, please don't hesitate to contact the author:

**Author:** Julep AI  
**Contact:** [hey@julep.ai](mailto:hey@julep.ai) or  <a href="https://discord.com/invite/JTSBGRZrzj" rel="dofollow">Discord</a>

Installing the Julep Client

In [17]:
!pip install julep -U --quiet

#### NOTE:

- UUIDs are generated for both the agent and task to uniquely identify them within the system.
- Once created, these UUIDs should remain unchanged for simplicity.
- Altering a UUID will result in the system treating it as a new agent or task.
- If a UUID is changed, the original agent or task will continue to exist in the system alongside the new one.

In [18]:
# Global UUID is generated for agent and task
import uuid

AGENT_UUID = uuid.uuid4()
TASK_UUID = uuid.uuid4() 

### Creating Julep Client with the API Key

In [20]:
from julep import Client

api_key = "" # Your API key here

# Create a client
client = Client(api_key=api_key, environment="dev")

## Creating an "agent"


Agent is the object to which LLM settings, like model, temperature along with tools are scoped to.

To learn more about the agent, please refer to the [documentation](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md#agent).

In [21]:
# Defining the agent
name = "Jarvis"
about = "The original AI conscious the Iron Man."
default_settings = {
    "temperature": 0.7,
    "top_p": 1,
    "min_p": 0.01,
    "presence_penalty": 0,
    "frequency_penalty": 0,
    "length_penalty": 1.0,
    "max_tokens": 150,
}

# Create the agent
agent = client.agents.create_or_update(
    agent_id=AGENT_UUID,
    name=name,
    about=about,
    model="gpt-4o",
)

## Defining a Task

Tasks in Julep are Github Actions style workflows that define long-running, multi-step actions. 
You can use them to conduct complex actions by defining them step-by-step. They have access to all Julep integrations.

To learn more about tasks, visit [Julep documentation](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md#task).

In [24]:
import yaml

Here is a Task which uses a agent to generate a sarcastic response to a given text using a DuckDuckGo and Wikipedia tool.

More on how to define a task can be found [here](https://github.com/julep-ai/julep/blob/dev/docs/julep-concepts.md).

In [25]:
task_def = yaml.safe_load("""
name: Tourist Plan With Weather And Attractions

input_schema:
  type: object
  properties:
    locations:
      type: array
      items:
        type: string
      description: The locations to search for.

tools:
- name: wikipedia
  type: integration
  integration:
    provider: wikipedia

- name: weather
  type: integration
  integration:
    provider: weather
    setup:
      openweathermap_api_key: "YOUR_API_KEY"

main:
- over: inputs[0].locations
  map:
    tool: weather
    arguments:
      location: _

- over: inputs[0].locations
  map:
    tool: wikipedia
    arguments:
      query: "_ + ' tourist attractions'"

- evaluate:
    zipped: "list(zip(inputs[0].locations, [output['result'] for output in outputs[0]], [output['documents'][0]['page_content'] for output in outputs[1]]))"  # [(location, weather, attractions)]


- over: _['zipped']
  parallelism: 3
  map:
    prompt:
    - role: system
      content: >-
        You are a travel assistant. Your task is to create a detailed itinerary for visiting tourist attractions in "{{_[0]}}" based on the weather conditions and the top tourist attractions provided.
        
        Current weather condition at "{{_[0]}}":
        "{{_[1]}}"

        Top tourist attractions in "{{_[0]}}":
        "{{_[2]}}"

        Suggest outdoor or indoor activities based on the above information.
    unwrap: true
""")

Creating/Updating a task to generate a sarcastic response to a given text using a Intergation.

In [26]:
# creating the task object
task = client.tasks.create_or_update(
    task_id=TASK_UUID,
    agent_id=AGENT_UUID,
    **task_def
)

## Creating an Execution

An execution is a single run of a task. It is a way to run a task with a specific set of inputs.

Creates a execution worflow for the Task defined in the yaml file.

In [28]:
execution = client.executions.create(
    task_id=task.id,
    input={
         "locations": ["New York", "London", "Paris", "Tokyo", "Sydney"]
    }
)

In [ ]:
execution.id

In [30]:
# getting the execution details
execution = client.executions.get(execution.id)
#printing the output
execution.output

Retrieves and lists all the steps of a defined task that have been executed up to that point in time. Unlike streaming, this function does not continuously monitor the execution; it only provides a snapshot of the steps completed so far without displaying real-time updates as the task progresses.

In [ ]:
client.executions.transitions.list(execution_id=execution.id).items

Continuously monitor and stream the steps of a defined task. It retrieves and displays the transitions or execution steps of the task identified by execution.id in real-time, showing each step sequentially until the task is either completed or an error causes it to terminate.

In [31]:
client.executions.transitions.stream(execution_id=execution.id)